### Lane detection and car detection


*   This code performs lane detection and object detection (car detection) on a video.

*  Lane Detection: Identifies lanes using edge detection (Canny), masking a region of interest, and finding lines using the Hough Line Transformation.
* Car Detection: Uses a pre-trained YOLOv8 model to detect cars in the video frame.
* Distance estimation is also calculated for each detected car using its bounding box dimensions.



### Install the required liberary

In [1]:
!pip install opencv-python-headless numpy ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 33.1 MB/s eta 0:00:00


In [5]:
import cv2
import numpy as np
import math
import time
from ultralytics import YOLO  # YOLOv8 module

# Configurable Parameters
CONFIG = {
    "input_video_path": "/content/car.mp4",  # Input video file path
    "output_video_path": "/content/test_1/detected_video.mp4",  # Output video file path
    "weights_path": "weights/yolov8n.pt",  # YOLOv8 weights file
    "frame_width": 1280,  # Output video width
    "frame_height": 720,  # Output video height
    "target_fps": 30,  # Target frames per second
    "lane_detection_canny_thresholds": (100, 200),  # Canny edge thresholds
    "lane_detection_hough_params": {
        "rho": 6,
        "theta": np.pi / 60,
        "threshold": 160,
        "min_line_length": 40,
        "max_line_gap": 25
    },
    "car_detection_confidence_threshold": 0.5  # Minimum confidence for car detection
}

# Function to mask out the region of interest
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    return cv2.bitwise_and(img, mask)

# Function to draw the filled polygon between the lane lines
def draw_lane_lines(img, left_line, right_line, color=[0, 255, 0], thickness=10):
    line_img = np.zeros_like(img)

    # Validate left_line and right_line
    if all(v == 0 for v in left_line) or all(v == 0 for v in right_line):
        # If either line is invalid, skip drawing
        return img

    # Create the polygon points
    poly_pts = np.array([[
        (left_line[0], left_line[1]),  # Bottom-left
        (left_line[2], left_line[3]),  # Top-left
        (right_line[2], right_line[3]),  # Top-right
        (right_line[0], right_line[1])  # Bottom-right
    ]], dtype=np.int32)

    # Ensure the polygon points are valid
    if poly_pts.shape[1] < 4:  # A valid polygon requires at least 4 points
        return img

    # Fill the polygon
    cv2.fillPoly(line_img, poly_pts, color)
    return cv2.addWeighted(img, 0.8, line_img, 0.5, 0.0)


# Lane detection pipeline
def pipeline(image, config):
    height, width = image.shape[:2]
    region_of_interest_vertices = [
        (0, height),
        (width / 2, height / 2),
        (width, height),
    ]

    # Convert to grayscale and apply Canny edge detection
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    cannyed_image = cv2.Canny(gray_image, *config["lane_detection_canny_thresholds"])

    # Mask out the region of interest
    cropped_image = region_of_interest(
        cannyed_image, np.array([region_of_interest_vertices], np.int32)
    )

    # Perform Hough Line Transformation to detect lines
    lines = cv2.HoughLinesP(
        cropped_image,
        rho=config["lane_detection_hough_params"]["rho"],
        theta=config["lane_detection_hough_params"]["theta"],
        threshold=config["lane_detection_hough_params"]["threshold"],
        minLineLength=config["lane_detection_hough_params"]["min_line_length"],
        maxLineGap=config["lane_detection_hough_params"]["max_line_gap"]
    )

    if lines is None:
        return image

    # Separate left and right lines
    left_line_x, left_line_y, right_line_x, right_line_y = [], [], [], []
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1) if (x2 - x1) != 0 else 0
            if math.fabs(slope) < 0.5:  # Ignore horizontal lines
                continue
            if slope <= 0:  # Left lane
                left_line_x.extend([x1, x2])
                left_line_y.extend([y1, y2])
            else:  # Right lane
                right_line_x.extend([x1, x2])
                right_line_y.extend([y1, y2])

    min_y, max_y = int(height * 3 / 5), height
    left_line = (
        [int(np.poly1d(np.polyfit(left_line_y, left_line_x, 1))(max_y)), max_y,
         int(np.poly1d(np.polyfit(left_line_y, left_line_x, 1))(min_y)), min_y]
        if left_line_x else [0, max_y, 0, min_y]
    )
    right_line = (
        [int(np.poly1d(np.polyfit(right_line_y, right_line_x, 1))(max_y)), max_y,
         int(np.poly1d(np.polyfit(right_line_y, right_line_x, 1))(min_y)), min_y]
        if right_line_x else [0, max_y, 0, min_y]
    )
    return draw_lane_lines(image, left_line, right_line)

# Function to estimate distance based on bounding box size
def estimate_distance(bbox_width):
    focal_length = 1000  # Example focal length, adjust as necessary
    known_width = 2.0  # Approximate car width in meters
    return (known_width * focal_length) / bbox_width

# Main function to process video
def process_video(config):
    model = YOLO(config["weights_path"])
    cap = cv2.VideoCapture(config["input_video_path"])

    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    # Set up video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(
        config["output_video_path"],
        fourcc,
        config["target_fps"],
        (config["frame_width"], config["frame_height"])
    )

    frame_time = 1.0 / config["target_fps"]

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (config["frame_width"], config["frame_height"]))
        lane_frame = pipeline(resized_frame, config)
        results = model(resized_frame)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0]
                cls = int(box.cls[0])
                if model.names[cls] == "car" and conf >= config["car_detection_confidence_threshold"]:
                    cv2.rectangle(lane_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                    bbox_width = x2 - x1
                    distance = estimate_distance(bbox_width)
                    cv2.putText(lane_frame, f'Distance: {distance:.2f}m', (x1, y2 + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        out.write(lane_frame)
        cv2.imshow("Lane and Car Detection", lane_frame)
        time.sleep(frame_time)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()




In [9]:
# Run the video processing function
# process_video(CONFIG)

#### This is not working as cv2.imshow dot work in colab notebook

In [10]:
import cv2
import numpy as np
import math
import time
from ultralytics import YOLO  # YOLOv8 module

# Configuration variables
CONFIG = {
    "input_video": "/content/car.mp4",       # Path to the input video
    "output_video": "/content/test_1/output.mp4",   # Path to save the processed video
    "weights": "weights/yolov8n.pt",       # YOLO model weights
    "output_resolution": (1280, 720),      # Output resolution (width, height)
    "fps": 30,                             # Frames per second for output video
    "confidence_threshold": 0.5,           # Minimum confidence to display bounding boxes
    "known_width": 2.0,                    # Approximate car width in meters
    "focal_length": 1000,                  # Focal length for distance estimation
}

# Function to mask out the region of interest
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

# Function to draw the filled polygon between the lane lines
def draw_lane_lines(img, left_line, right_line, color=[0, 255, 0], thickness=10):
    line_img = np.zeros_like(img)
    if all(v == 0 for v in left_line) or all(v == 0 for v in right_line):
        return img

    poly_pts = np.array([[
        (left_line[0], left_line[1]),
        (left_line[2], left_line[3]),
        (right_line[2], right_line[3]),
        (right_line[0], right_line[1])
    ]], dtype=np.int32)

    if poly_pts.shape[1] < 4:
        return img

    cv2.fillPoly(line_img, poly_pts, color)
    return cv2.addWeighted(img, 0.8, line_img, 0.5, 0.0)

# The lane detection pipeline
def pipeline(image):
    height, width = image.shape[:2]
    region_of_interest_vertices = [
        (0, height),
        (width / 2, height / 2),
        (width, height),
    ]

    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    cannyed_image = cv2.Canny(gray_image, 100, 200)
    cropped_image = region_of_interest(cannyed_image, np.array([region_of_interest_vertices], np.int32))

    lines = cv2.HoughLinesP(
        cropped_image, rho=6, theta=np.pi / 60, threshold=160,
        lines=np.array([]), minLineLength=40, maxLineGap=25
    )

    left_line_x, left_line_y, right_line_x, right_line_y = [], [], [], []

    if lines is None:
        return image

    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1) if (x2 - x1) != 0 else 0
            if math.fabs(slope) < 0.5:
                continue
            if slope <= 0:
                left_line_x.extend([x1, x2])
                left_line_y.extend([y1, y2])
            else:
                right_line_x.extend([x1, x2])
                right_line_y.extend([y1, y2])

    min_y = int(height * (3 / 5))
    max_y = height

    left_x_start, left_x_end, right_x_start, right_x_end = 0, 0, 0, 0
    if left_line_x and left_line_y:
        poly_left = np.poly1d(np.polyfit(left_line_y, left_line_x, deg=1))
        left_x_start, left_x_end = int(poly_left(max_y)), int(poly_left(min_y))

    if right_line_x and right_line_y:
        poly_right = np.poly1d(np.polyfit(right_line_y, right_line_x, deg=1))
        right_x_start, right_x_end = int(poly_right(max_y)), int(poly_right(min_y))

    return draw_lane_lines(image, [left_x_start, max_y, left_x_end, min_y], [right_x_start, max_y, right_x_end, min_y])

# Function to estimate distance based on bounding box size
def estimate_distance(bbox_width, bbox_height, config):
    known_width = config["known_width"]
    focal_length = config["focal_length"]
    return (known_width * focal_length) / bbox_width if bbox_width else 0

# Main function to read and process video with YOLOv8
def process_video(config):
    model = YOLO(config["weights"])
    cap = cv2.VideoCapture(config["input_video"])

    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(config["output_video"], fourcc, config["fps"], config["output_resolution"])

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, config["output_resolution"])
        lane_frame = pipeline(resized_frame)
        results = model(resized_frame)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0]
                cls = int(box.cls[0])

                if model.names[cls] == 'car' and conf >= config["confidence_threshold"]:
                    label = f'{model.names[cls]} {conf:.2f}'
                    cv2.rectangle(lane_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                    cv2.putText(lane_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                    distance = estimate_distance(x2 - x1, y2 - y1, config)
                    distance_label = f'Distance: {distance:.2f}m'
                    cv2.putText(lane_frame, distance_label, (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        out.write(lane_frame)

    cap.release()
    out.release()
    print(f"Processed video saved at {config['output_video']}")

# Run the video processing function
process_video(CONFIG)



0: 384x640 6 cars, 161.0ms
Speed: 2.5ms preprocess, 161.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 166.7ms
Speed: 3.0ms preprocess, 166.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 146.2ms
Speed: 2.7ms preprocess, 146.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 166.9ms
Speed: 3.4ms preprocess, 166.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 154.9ms
Speed: 2.6ms preprocess, 154.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 144.3ms
Speed: 2.9ms preprocess, 144.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 172.1ms
Speed: 2.6ms preprocess, 172.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 143.3ms
Speed: 4.1ms preprocess, 143.3ms inference, 1.5ms postprocess p